In [1]:
import bmdrc
import pandas as pd

## Module 1: Input Data Modules 

##### LPR Class

In [7]:
from bmdrc.LPRClass import LPRClass

lpr_example = pd.read_csv("/Users/degn400/Git_Repos/bmdrc/data/LPR_Long.csv")

LPR = LPRClass(
    df = lpr_example,
    chemical = "chemical.id",
    plate = "plate.id",
    well = "well",
    concentration = "conc",
    time = "variable",
    value = "value",
    cycle_length = 30
)

LPR

TypeError: expected string or bytes-like object, got 'Series'